# 이상치 탐지

## 불균형 데이터 셋

### 참조 : https://dacon.io/competitions/official/235894/codeshare/4971?page=2&dtype=recent

#### 라이브러리 임포트하기

In [7]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2 as cv
import timm
import os
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

## 데이터 준비

In [9]:
train_y = pd.read_csv("C:/Users/choij/Desktop/datasets/open/open/train_df.csv")
train_labels = train_y['label']

train_y.head() 

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good


## 라벨 전처리

In [10]:
label_unique = sorted(np.unique(train_labels)) # 88개의 라벨
label_unique = {key:value for key, value in zip(label_unique, range(len(label_unique)))}
# 각 라벨 별로 번호 붙이기. 88개의 라벨 있음을 확인
train_labels = [label_unique[k] for k in train_labels]
len(train_labels) # 4277개의 훈련 데이터

4277

In [11]:
train_dir = "C:/Users/choij/Desktop/datasets/open/open/train/train"
test_dir = "C:/Users/choij/Desktop/datasets/open/open/test/test"

## 이미지 전처리

In [19]:
train_png = sorted(glob(train_dir + '/*.png'))  # 폴더 안에 있는 모든 이미지들 리스트에 넣음
test_png =  sorted(glob(test_dir + '/*.png'))

In [20]:
len(train_png) # 4277개의 이미지 리스트

4277

In [21]:
train_img = [] # 새로운 이미지 리스트
test_img = []  # 주소에 맞게 replace 활용해서 변환
for i in train_png:
    img = i.replace('\\', '/')
    train_img.append(img)

for i in test_png:
    img = i.replace('\\', '/')
    test_img.append(img) 

In [22]:
def img_resize(path): # 이미지 크기 변환
    img = cv.imread(path)[:,:,::-1] # 마지막 순서 바꾸는 이유 : RGB형태로 변환 위해(처음에는 BGR)
    img = cv.resize(img, (224, 224)) # 크기 (224, 224) 로 변환
    return img

In [23]:
train_img = [img_resize(img) for img in train_img] #이미지 변환 함수 적용
test_img = [img_resize(img) for img in test_img]

In [24]:
print(train_img[0].shape) #(224,224,3) 의 이미지로 변환되었음을 확인
print(test_img[0].shape)

(224, 224, 3)
(224, 224, 3)


## 모델 정의 및 데이터 증강

In [25]:
class Custom_dataset(Dataset): #파이토치의 Dataset 클래스 상속받는 나만의 데이터셋 정의
    def __init__(self, img_path, labels, mode = 'train'): 
        self.img_path = img_path  # 기존 이미지와 라벨들 정의
        self.labels = labels
        self.mode = mode # 기본값을 train으로 받는다.
    
    def __len__(self):
        return len(self.img_path)
    
    def __getitem__(self, idx):
        img = self.img_path[idx] #각각의 이미지 불러들임
        if self.mode == 'train': #만약 들어오는 데이터셋이 훈련 데이터라면 
            augmentation = random.randint(0,8) # 데이터 증강을 위해 따로 정의한 코드같다.
            if augmentation < 3:
                pass
            
            elif augmentation == 3:
                img = cv.rotate(img, cv.ROTATE_90_CLOCKWISE)
            
            elif augmentation == 4:
                img = cv.rotate(img, cv.ROTATE_90_CLOCKWISE)
            
            elif augmentation == 5:
                img = img[::-1].copy()
                
            elif augmentation == 6:
                img = img[:,::-1].copy()
            
            elif augmentation == 7:
                img = img[::-1,::-1,:].copy()
        
        img = transforms.ToTensor()(img)
        if self.mode == 'test': #만약 훈련 데이터셋이 아니라면 데이터증강은 하지 않는다.
            pass
        
        label = self.labels[idx]
        
        return img, label # 각각의 이미지와 라벨을 리턴한다.

### Custom Dataset 사용하는 이유:

#### 방대한 양의 데이터를 한 번에 불러오기 쉽지 않기 때문에 데이터를 하나씩만 불러서 쓰는 방식 써야함

#### 모든 데이터 불러놓고 사용하는 기존 Dataset 말고 Custom Dataset 필요

#### from troch.utils.data import Dataset 통해 불러온 후 x와 y를 tensor 로 넣어주면 됨

#### 파이토치의 dataset : 샘플과 정답을 저장하고

#### dataloader 쉽게 접근할 수 있도록 순회 가능한(iterable)객체로 감싼다

#### Custom Dataset 만들 때 __getitem__, __len__, __init__ 반드시 구현

In [26]:
import torchvision

In [27]:
train_dir = 'C:/Users/choij/Desktop/datasets/open/open/train/small_train'
train_dir

'C:/Users/choij/Desktop/datasets/open/open/train/small_train'

In [ ]:
##############################

### 만약 이미지들을 각 폴더로 저장하고 깔끔하게 분류했다면 아래와 같은 방법도 사용 가능

In [42]:
transf = transforms.Compose([transforms.Resize(224), transforms.ToTensor()]) # 224 로 크기 설정, tensor로 변환하는 양식 저장

trainset = torchvision.datasets.ImageFolder(root = train_dir, transform = transf) #이미지들 있는 폴더, 변환 양식
trainloader = DataLoader(trainset, batch_size = 5, shuffle = False, num_workers = 2) # trainloader 로 배치사이즈, 셔플유무 결정

len(trainloader)

685

In [44]:
trainset[0][0].size() # 이런 식으로 dataloader 이용하여 전처리 가능

torch.Size([3, 224, 224])

In [ ]:
####################################

In [30]:
class Model(nn.Module): # nn.Module 상속
    def __init__(self):
        super(Model, self).__init__() ## Module의 init 상속
        self.model = timm.create_model('efficientnet_b3', #timm 라이브러리 이용하여 훈련된 데이터셋 가져온다.
                                       pretrained = True, num_classes = len(label_unique))
    def forward(self, x):
        x = self.model(x) # 훈련된 데이터셋을 더한다.
        return x

In [28]:
batch_size = 64  # 배치 사이즈 결정

train_dataset = Custom_dataset(np.array(train_img), np.array(train_labels), mode = 'train') #훈련 데이터
train_loader = DataLoader(train_dataset, shuffle = True, batch_size = batch_size) #훈련 dataloader

test_dataset = Custom_dataset(np.array(test_img), np.array(['tmp'] * len(test_img)), mode = 'test') #실험데이터
test_loader = DataLoader(test_dataset, shuffle = False, batch_size = batch_size)#실험 dataloader

In [31]:
model = Model() # 모델 객체 생성

optimizer = torch.optim.AdamW(model.parameters(), lr = 1.0e-4)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() #amp :automatic mixed 

### amp : Automatic Mixed Precision 줄임말

### amp를 쓰면 gpu메모리 소모 줄고 연산속도가 빨라진다

### 이유: 행렬 곱을 더 효율적으로 하게 만든다.

### ex.다른 행렬 끼리의 연산에서도 dtype을 변환해주어 신경망의 runtime 과 memory줄인다.

## 정확도 점수 함수 정의

In [32]:
def score_function(real, pred):
    score = f1_score(real, pred, average = 'macro')
    return score

In [39]:
epochs = 3 #3번 반복(cpu라서 시간이 너무 오래걸린다)

for epoch in range(epochs):
    start = time.time() #시간 측정
    train_loss = 0 #훈련 손실값 각 에포크 마다 정의
    train_pred = []#훈련 예측값 
    train_y = [] #
    model.train() # 모델이 훈련할 수 있도록 train mode 로 변환
    
    for batch in (train_loader): # 각 배치사이즈마다 훈련시작.여기서는 64
        optimizer.zero_grad() #기울기 초기화
        x = torch.tensor(batch[0], dtype = torch.float32) #각 batch[0]은 x데이터, batch[1]은 y데이터이다.
        y = torch.tensor(batch[1], dtype = torch.long)
        with torch.cuda.amp.autocast():
            pred = model(x) # x값에 대해 pred값 저장(출력값이 softmax이다.)
        loss = criterion(pred, y) # pred값과 실제 y값에 대해서 crossentropy 손실함수 측정
        
        scaler.scale(loss).backward() # 손실에 대하여 역전파 실행하기
        scaler.step(optimizer) #최적화 함수 적용하기. 여기서는 Adam 적용
        scaler.update() # 가중치 업데이트
        # 모든 계산들은 amp를 통해 이루어지는 걸 확인할 수 있다.
        
        train_loss += loss.item() / len(train_loader) #로스 값 각 에포크마다 저장해주기
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist() # 예측값에서 가장 높은 값 리스트에 저장하기
        train_y += y.detach().cpu().numpy().tolist() # 실제 y값 복사
        
    train_f1 = score_function(train_y, train_pred) # 예측값과 실제 y값과 비교를 통해 정확도 산출
    
    Time = time.time() - start
    
    print(f'epoch : {epoch+1}/{epochs}    time : {Time:.0f}s/{Time*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

epoch : 1/3    time : 2281s/4561s
TRAIN    loss : 0.30766    f1 : 0.61397
epoch : 2/3    time : 2258s/2258s
TRAIN    loss : 0.22233    f1 : 0.70943
epoch : 3/3    time : 2260s/0s
TRAIN    loss : 0.18242    f1 : 0.77142


## 3번의 훈련 끝에 0.77의 정확도가 나왔음을 알 수 있다.

## 테스트해보기

In [41]:
model.eval() #모델 test 모드로 변경
f_pred = [] #테스트 예측값 담아줄 리스트

with torch.no_grad(): #테스트 데이터이므로 미분x
    for batch in (test_loader): #테스트 dataloader반복문 통해 각 batch불러온다
        x = torch.tensor(batch[0], dtype = torch.float32, device = 'cpu') #
        with torch.cuda.amp.autocast():
            pred = model(x) #x값에 대해 예측 값(softmax값이 들어가있으므로 각 클래스에 속할 확률들이 들어가있.)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist()) #각 확률값에서 가장 높은 값 선택해서 리스트로 넣어준다.

In [44]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

submission = pd.read_csv("C:/Users/choij/Desktop/datasets/open/open/sample_submission.csv")

submission['label'] = f_result

submission.to_csv('submission.csv', index = False)

In [55]:
su = pd.read_csv('C:/Users/choij/submission.csv')
su['label']

0        tile-glue_strip
1              grid-good
2        transistor-good
3       tile-gray_stroke
4              tile-good
              ...       
2149    tile-gray_stroke
2150          screw-good
2151           grid-good
2152          cable-good
2153         zipper-good
Name: label, Length: 2154, dtype: object